In [30]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import albumentations as A
from sklearn.model_selection import train_test_split
from albumentations import (
    HorizontalFlip, VerticalFlip, RandomRotate90,
    ShiftScaleRotate, Blur, RandomBrightnessContrast,
    Compose
)


In [ ]:
data_dir = r'C:\Users\haris\ArtificialIntelligence\MachineLearning\Projects\Angry Sad Happy'

# Create a list to store data
images = []
labels = []

# Define the augmentation transformations
augmentation_transform = Compose([
    HorizontalFlip(p=0.5),            # Horizontal flip with 50% probability
    VerticalFlip(p=0.5),              # Vertical flip with 50% probability
    RandomRotate90(p=0.5),            # Random rotation by 90 degrees with 50% probability
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),  # Random shift, scale, and rotate
    Blur(p=0.2),                      # Blur with 20% probability
    RandomBrightnessContrast(p=0.3),  # Random brightness and contrast adjustment with 30% probability
])

# Create a list to store augmented data
augmented_images = []
augmented_labels = []

# Loop through all folders and images
for emotion in ['Happy', 'Sad', 'Angry']:
    folder_path = os.path.join(data_dir, emotion)
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Apply data augmentation to the image
        for _ in range(100):
            augmented = augmentation_transform(image=image)
            augmented_image = augmented['image']
            augmented_images.append(augmented_image)
            augmented_labels.append(emotion)

# Convert the list of augmented images and labels to numpy arrays
X_augmented = np.array(augmented_images)
y_augmented = np.array(augmented_labels)

# # Convert the list of images and labels to numpy arrays
# X = np.array(images)
# y = np.array(labels)

# Encode labels into numeric form
label_to_numeric = {'Happy': 0, 'Sad': 1, 'Angry': 2}
y_numeric = np.array([label_to_numeric[label] for label in y_augmented])

# Convert labels to one-hot encoded format
y_one_hot = to_categorical(y_numeric, num_classes=3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_one_hot, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 output neurons for the 3 emotions

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)


In [ ]:
# Assuming you have a single image as 'test_image'
test_image = cv2.imread(r'C:\Users\haris\ArtificialIntelligence\MachineLearning\Projects\Angry Sad Happy\73705bd7debb66c2afc780a22c223804.jpg')
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image = cv2.resize(test_image, (128, 128))
test_image = np.expand_dims(test_image, axis=0)  # Add a batch dimension

predicted_probs = model.predict(test_image)
predicted_emotion = ['Happy', 'Sad', 'Angry'][np.argmax(predicted_probs)]
print("Predicted emotion:", predicted_emotion)
